<a href="https://colab.research.google.com/github/fvillena/dcc-ia-nlp/blob/master/8_transformers_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Los siguientes ejemplos ha sido basados en los ejemplos de 
https://github.com/huggingface/transformers



In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 5.3MB/s 
     |████████████████████████████████| 2.9MB 40.4MB/s 
     |████████████████████████████████| 1.1MB 38.3MB/s 
     |████████████████████████████████| 890kB 39.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f5b7c496aa846938a99ba650a816d3323bcf94bd46c676b71dfb06e66ce90fe6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import pipeline

#Responder pregunta segun el contexto
nlp = pipeline("question-answering")

context = r"""
... 
... Los datos han pasado a constituir parte esencial de cualquier empresa, desde el ámbito del entretenimiento personal hasta 
... los negocios y la planificación estratégica de grandes organizaciones. Las tecnologías de captura, procesamiento y 
... comunicación de datos los han puesto en primer plano. La noción de “diluvio de datos” hace referencia a esa marea 
... aparentemente incontenible de datos que rodean (y muchas veces ahogan) a las personas y organizaciones hoy.
... Para analizar y sacar provecho de este diluvio de datos, los profesionales requieren tener conocimientos tanto 
... teóricos como prácticos de lo que se conoce como Ciencia de los Datos, que incluye áreas como Minería de Datos, 
... Recuperación de Información, Visualización y Gestión de Datos. El presente Diplomado apunta a formar profesionales 
... especialistas que trabajen como analistas expertos en el manejo de datos complejos y masivos..
... """

result = nlp(question="¿Qué conocimientos se imparten en el diplomado?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


Answer: 'El presente Diplomado apunta a formar profesionales', score: 0.0001, start: 784, end: 836


In [ ]:
#clasificar si es positivo o negativo
from transformers import pipeline

nlp = pipeline("sentiment-analysis")

result = nlp("No me molestó")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

result = nlp("A nadie le gustó")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9945
label: POSITIVE, with score: 0.999


In [ ]:
#generacion de texto
from transformers import AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("xlnet-base-cased", return_dict=True)
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology

PADDING_TEXT =""" Los datos han pasado a constituir parte esencial de cualquier empresa, desde el ámbito del entretenimiento personal hasta 
... los negocios y la planificación estratégica de grandes organizaciones. Las tecnologías de captura, procesamiento y 
... comunicación de datos los han puesto en primer plano. La noción de “diluvio de datos” hace referencia a esa marea 
... aparentemente incontenible de datos que rodean (y muchas veces ahogan) a las personas y organizaciones hoy.
... Para analizar y sacar provecho de este diluvio de datos, los profesionales requieren tener conocimientos tanto 
... teóricos como prácticos de lo que se conoce como Ciencia de los Datos, que incluye áreas como Minería de Datos, 
... Recuperación de Información, Visualización y Gestión de Datos. El presente Diplomado apunta a formar profesionales 
... especialistas que trabajen como analistas expertos en el manejo de datos complejos y masivos. <eod> </s> <eos>
... """

prompt = "Los datos en mi empresa "
inputs = tokenizer.encode(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]

print(generated)

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Los datos en mi empresa i empresa


In [ ]:
#resumir textos
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ Los datos han pasado a constituir parte esencial de cualquier empresa, desde el ámbito del entretenimiento personal hasta 
... los negocios y la planificación estratégica de grandes organizaciones. Las tecnologías de captura, procesamiento y 
... comunicación de datos los han puesto en primer plano. La noción de “diluvio de datos” hace referencia a esa marea 
... aparentemente incontenible de datos que rodean (y muchas veces ahogan) a las personas y organizaciones hoy.
... Para analizar y sacar provecho de este diluvio de datos, los profesionales requieren tener conocimientos tanto 
... teóricos como prácticos de lo que se conoce como Ciencia de los Datos, que incluye áreas como Minería de Datos, 
... Recuperación de Información, Visualización y Gestión de Datos. El presente Diplomado apunta a formar profesionales 
... especialistas que trabajen como analistas expertos en el manejo de datos complejos y masivos. <eod> </s> <eos>
... """


print(summarizer(ARTICLE, max_length=5, min_length=4, do_sample=False))

[{'summary_text': ' Los datos han pasado a parte esencial de cualqu'}]
